In [ ]:
import os, re
import urllib.request, json 

import tqdm
import time

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import optuna
import pickle as pkl

from joblib import Parallel, delayed

In [ ]:
# from EnvDL.core import *
from dataG2F.qol import ensure_dir_path_exists
cache_path = '../nbs_artifacts/10_collect_historical_data/'
ensure_dir_path_exists(dir_path = cache_path)

Sector : CROPS
Group: FIELD CROPS 
    Commodity: CORN
    Category: AREA HARVESTED, YIELD
    Data Item: CORN, GRAIN, *
    
Select Location:
Geographic Level: COUNTY
State: # as needed to be under the download limit

Select Time:
Year: *-2022
Period Type: ANNUAL

https://quickstats.nass.usda.gov/#816634B5-2E9C-335E-B52C-FDD998ACE7D0

Alabama - Florida
Georgia - Illinois
Indiana - 
Iowa -
Kansas - 
Kentucky - Massachusetts
Michigan - Mississippi
Missouri - Montana
Nebraska (Ag District Central - Southeast)
Nebraska (Ag District Southwest - Southwest)
Nevada - North Carolina
North Dakota - Ohio
Oklahoma - South Carolina
South Dakota - Tennessee
Texas - West Virginia
Wisconsin - Wyoming

In [ ]:
load_from = '../data_ext/zma/nass/'

nass_csvs = [e for e in os.listdir(load_from) if re.match('[A-Z0-9]{8}\-[A-Z0-9]{4}\-[A-Z0-9]{4}\-[A-Z0-9]{4}\-[A-Z0-9]{12}\.csv', e)]
# nass_csvs = [e for e in nass_csvs if e != nass_state_csv]

nass_df = pd.concat([pd.read_csv(load_from+nass_csv, low_memory=False) for nass_csv in nass_csvs])
nass_df.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2017,YEAR,NaN,COUNTY,MICHIGAN,26,CENTRAL,50,CLARE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - ACRES HARVESTED",TOTAL,NOT SPECIFIED,"2,337",17.0
1,CENSUS,2017,YEAR,NaN,COUNTY,MICHIGAN,26,CENTRAL,50,CLARE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - OPERATIONS WITH AREA HARVESTED",AREA HARVESTED,AREA HARVESTED: (1.0 TO 24.9 ACRES),38,55.1
2,CENSUS,2017,YEAR,NaN,COUNTY,MICHIGAN,26,CENTRAL,50,CLARE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - OPERATIONS WITH AREA HARVESTED",AREA HARVESTED,AREA HARVESTED: (100 TO 249 ACRES),3,44.8
3,CENSUS,2017,YEAR,NaN,COUNTY,MICHIGAN,26,CENTRAL,50,CLARE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - OPERATIONS WITH AREA HARVESTED",AREA HARVESTED,AREA HARVESTED: (25.0 TO 99.9 ACRES),6,36.1
4,CENSUS,2017,YEAR,NaN,COUNTY,MICHIGAN,26,CENTRAL,50,CLARE,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - OPERATIONS WITH AREA HARVESTED",AREA HARVESTED,AREA HARVESTED: (250 TO 499 ACRES),4,37.0


In [ ]:
nass_df = nass_df.loc[:, [
#     'Program',
    'Year',
#     'Period',
#     'Week Ending',
#     'Geo Level',
    'State',
#     'State ANSI',
    'Ag District',
#     'Ag District Code',
    'County',
#     'County ANSI',
#     'Zip Code',
#     'Region',
#     'watershed_code',
#     'Watershed',
#     'Commodity',
    'Data Item',
#     'Domain',
#     'Domain Category',
    'Value',
#     'CV (%)'
]]

[[len(set(nass_df[e])), e] for e in list(nass_df)]

[[113, 'Year'],
 [49, 'State'],
 [87, 'Ag District'],
 [1721, 'County'],
 [11, 'Data Item'],
 [24997, 'Value']]

In [ ]:


data_dict = {
                "CORN, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / ACRE": "GRN_IRR_BUpACRE",
               "CORN, GRAIN - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_BUpNETPLANTEDACRE",
"CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_NON_IRR_BUpNETPLANTEDACRE",
             "CORN, SILAGE, IRRIGATED - YIELD, MEASURED IN TONS / ACRE": "SLG_IRR_TONSpACRE",
         "CORN, SILAGE, NON-IRRIGATED - YIELD, MEASURED IN TONS / ACRE": "SLG_NON_IRR_TONSpACRE",
            "CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE": "GRN_NON_IRR_BUpACRE",
    "CORN, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_IRR_BUpNETPLANTEDACRE",
                        "CORN, SILAGE - YIELD, MEASURED IN TONS / ACRE": "SLG_TONSpACRE",
                           "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE": "GRN_BUpACRE"
}



In [ ]:
nass_df = nass_df.merge(pd.DataFrame(data_dict, index = [0]).T.reset_index(
                ).rename(columns = {'index':'Data Item', 0:'Key'})
                ).drop(columns = ['Data Item'])
nass_df = nass_df.rename(columns = {'Ag District': 'AgDistrict'})



In [ ]:
nass_df

,Year,State,AgDistrict,County,Value,Key
0,2022,MICHIGAN,NaN,OTHER COUNTIES,163.1,GRN_BUpACRE
1,2022,MICHIGAN,CENTRAL,CLARE,138.6,GRN_BUpACRE
2,2022,MICHIGAN,CENTRAL,GLADWIN,147.1,GRN_BUpACRE
3,2022,MICHIGAN,CENTRAL,GRATIOT,170.9,GRN_BUpACRE
4,2022,MICHIGAN,CENTRAL,ISABELLA,160.8,GRN_BUpACRE
...,...,...,...,...,...,...
244597,1972,NEBRASKA,SOUTHWEST,HITCHCOCK,41.7,GRN_NON_IRR_BUpNETPLANTEDACRE
244598,1972,NEBRASKA,SOUTHWEST,KEITH,23.6,GRN_NON_IRR_BUpNETPLANTEDACRE
244599,1972,NEBRASKA,SOUTHWEST,LINCOLN,45.4,GRN_NON_IRR_BUpNETPLANTEDACRE
244600,1972,NEBRASKA,SOUTHWEST,PERKINS,29.1,GRN_NON_IRR_BUpNETPLANTEDACRE


In [ ]:


nass_df_wide = nass_df.pivot(columns='Key', 
                             values='Value', 
                             index=['Year', 'State', 'AgDistrict', 'County']
                            ).reset_index()
nass_df_wide



Key,Year,State,AgDistrict,County,GRN_BUpACRE,GRN_BUpNETPLANTEDACRE,GRN_IRR_BUpACRE,GRN_IRR_BUpNETPLANTEDACRE,GRN_NON_IRR_BUpACRE,GRN_NON_IRR_BUpNETPLANTEDACRE
0,1910,NEBRASKA,CENTRAL,BUFFALO,24,NaN,NaN,NaN,NaN,NaN
1,1910,NEBRASKA,CENTRAL,CUSTER,21.4,NaN,NaN,NaN,NaN,NaN
2,1910,NEBRASKA,CENTRAL,DAWSON,17.1,NaN,NaN,NaN,NaN,NaN
3,1910,NEBRASKA,CENTRAL,GREELEY,24,NaN,NaN,NaN,NaN,NaN
4,1910,NEBRASKA,CENTRAL,HALL,31.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
187903,2022,WISCONSIN,WEST CENTRAL,MONROE,162.4,NaN,NaN,NaN,NaN,NaN
187904,2022,WISCONSIN,WEST CENTRAL,PEPIN,171.9,NaN,NaN,NaN,NaN,NaN
187905,2022,WISCONSIN,WEST CENTRAL,PIERCE,181.2,NaN,NaN,NaN,NaN,NaN
187906,2022,WISCONSIN,WEST CENTRAL,ST CROIX,165.9,NaN,NaN,NaN,NaN,NaN


In [ ]:


# drop cols with low fill rate
nass_df_wide = nass_df_wide.drop(columns = [
    'GRN_BUpNETPLANTEDACRE',
    'GRN_IRR_BUpNETPLANTEDACRE',
    'GRN_NON_IRR_BUpNETPLANTEDACRE'
])



In [ ]:
# nass_df_wide.loc[nass_df_wide.Year == 2022, ]

# add a placeholder for 2022 data

tmp = nass_df_wide.loc[:, ['State', 'AgDistrict', 'County']].drop_duplicates().reset_index().drop(columns = ['index'])
tmp['Year'] = 2022

nass_df_wide = nass_df_wide.merge(tmp, how = 'outer')

In [ ]:
nass_df_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189565 entries, 0 to 189564
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Year                 189565 non-null  int64 
 1   State                189565 non-null  object
 2   AgDistrict           189472 non-null  object
 3   County               189565 non-null  object
 4   GRN_BUpACRE          187413 non-null  object
 5   GRN_IRR_BUpACRE      16628 non-null   object
 6   GRN_NON_IRR_BUpACRE  14745 non-null   object
dtypes: int64(1), object(6)
memory usage: 10.1+ MB


In [ ]:
nass_df_wide

Key,Year,State,AgDistrict,County,GRN_BUpACRE,GRN_IRR_BUpACRE,GRN_NON_IRR_BUpACRE
0,1910,NEBRASKA,CENTRAL,BUFFALO,24,NaN,NaN
1,1910,NEBRASKA,CENTRAL,CUSTER,21.4,NaN,NaN
2,1910,NEBRASKA,CENTRAL,DAWSON,17.1,NaN,NaN
3,1910,NEBRASKA,CENTRAL,GREELEY,24,NaN,NaN
4,1910,NEBRASKA,CENTRAL,HALL,31.1,NaN,NaN
...,...,...,...,...,...,...,...
189560,2022,IOWA,NORTHWEST,OTHER (COMBINED) COUNTIES,NaN,NaN,NaN
189561,2022,IOWA,WEST CENTRAL,OTHER (COMBINED) COUNTIES,NaN,NaN,NaN
189562,2022,MINNESOTA,SOUTH CENTRAL,OTHER (COMBINED) COUNTIES,NaN,NaN,NaN
189563,2022,OHIO,NORTHWEST,OTHER (COMBINED) COUNTIES,NaN,NaN,NaN


In [ ]:
# Filter to have the most y obs and match nasa power data
mask = ((nass_df_wide.Year >= 1981) &
        (nass_df_wide.GRN_BUpACRE.notna()))


nass_df_wide = nass_df_wide.loc[mask, ]
# Filter to exclude data during G2F
mask = (nass_df_wide.Year < 2014)
nass_df_wide = nass_df_wide.loc[mask, ]

In [ ]:
nass_df_wide.to_csv(cache_path+'nass_historical.csv', index=False)